In [1]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py


# plt.rcParams["figure.figsize"] = (24,16)
# plt.rcParams["font.size"] = 24
# plt.rcParams['xtick.labelsize'] = "small"
# plt.rcParams['ytick.labelsize'] = 36

In [2]:
%matplotlib widget
#%reload_ext autoreload
#%autoreload 2

### Discharge data

In [3]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def func1_scale(i):
    bb = step_cal_files[i]
        
    freq2 = np.fft.rfftfreq(len(bb.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(bb.electrode_data[0]))
    phi_tmp2 = (fft_angles[freq2==drive_freq]+np.pi/2)%(2*np.pi)
    print(phi_tmp2,np.std(bb.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': phi_tmp2, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [215]:
step_cal_dir = r'/data/new_trap/20210830/Bead1/Discharge/FineDischarge/'
step_cal_files = load_dir(step_cal_dir, start_file=0)

100%|██████████| 185/185 [00:04<00:00, 40.28it/s]

185  files in folder
185  files loaded


In [216]:
amps3 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    amps3.append(func1_scale(i))

0 / 185
[0.07216374] 0.05004696352452862
1 / 185
[0.07218011] 0.05004674261804184
2 / 185
[0.07217364] 0.05004664610410522
3 / 185
[0.07219365] 0.05004697284574258
4 / 185
[0.07219079] 0.050046528696682864
5 / 185
[0.07219249] 0.050046733284185005
6 / 185
[0.07221319] 0.05004681596322339
7 / 185
[0.07223124] 0.05004672905591215
8 / 185
[0.07222904] 0.05004656813982753
9 / 185
[0.07222753] 0.05004692924467627
10 / 185
[0.07222463] 0.050046983363467364
11 / 185
[0.07225377] 0.050046594234892855
12 / 185
[0.07225198] 0.050046846698829875
13 / 185
[0.07223048] 0.05004707515321758
14 / 185
[0.07223096] 0.05004684113100025
15 / 185
[0.07213118] 0.050046489889492105
16 / 185
[0.0721514] 0.05004654773486393
17 / 185
[0.07216683] 0.05004690868597997
18 / 185
[0.07216085] 0.050046393547231255
19 / 185
[0.07216514] 0.05004671808595284
20 / 185
[0.07216004] 0.05004681631103741
21 / 185
[0.07215716] 0.050046393634943384
22 / 185
[0.07216653] 0.05004636430981544
23 / 185
[0.07217415] 0.0500467585547

In [252]:
scale = np.abs(np.mean(amps3[150:180]))/1
print("Scale is ", scale)
# scale *= -1
bb = step_cal_files[-1]
voltage = np.std(bb.electrode_data[0])*np.sqrt(2)*200
print("voltage: ", voltage)

Scale is  4.43129187912857e-05
voltage:  10.009457129765254


In [234]:
elec_i = [bb.electrode_channels[1] for bb in step_cal_files]

In [243]:
# amps = np.array(amps)
amps3 = np.array(amps3)
_,ax = plt.subplots(figsize=(9.5,4))
# scale = 3.5e-5

# Perrinf
ax.scatter(range(len(amps3)), amps3/scale, label='fit phase')
# ax.scatter(range(len(amps3)), elec_i, label='electrode index')

ax.set(xlabel='dataset', ylabel='Electron charges', ylim=(-3,12), xlim=(100,None))
ax.grid(axis='y')
ax.minorticks_on()
ax.grid(which='major', linestyle='-', linewidth='0.5', color='red')
ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
ax.legend()
from matplotlib.ticker import MaxNLocator
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [244]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp(i):
    
    fname = r'/data/new_trap/20210830/Bead1/Neutrality/20V_100V_YZspin_98p98kHz_20210903/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 'sigma': 0.1,
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1, 'error_sigma':0.1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 'limit_sigma': [0, 1],
                  'limit_A': [-100, 100], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0], m1_tmp.values[3]

In [245]:
i=10
get_amp(i)/scale

array([ 0.02077442, 13.53623047])

In [164]:
i = 4
fname = r'/data/new_trap/20210830/Bead1/Neutrality/200V_100V_YZspin_98p98kHz_20210902/Discharge_'+str(i)+'.h5'
bb = BeadDataFile.BeadDataFile(fname=fname)
voltage0_200 = np.std(bb.electrode_data[0])*np.sqrt(2)*200
voltage1_200 = np.std(bb.electrode_data[1])*np.sqrt(2)*200
bias2 = np.mean(bb.electrode_data[0])*200
print("voltages, bias: ", voltage0_200, voltage1_200, bias2)

voltages, bias:  200.0415044870712 0.03907095035386525 0.4126561574727817


In [176]:
voltage, voltage0_20, voltage0_200

(9.996262322298485, 20.00907917617504, 200.0415044870712)

In [246]:
amps_data_20V_2 = np.array(Parallel(n_jobs=20)(delayed(get_amp)(i) for i in tqdm(range(1300))))

100%|██████████| 1300/1300 [00:19<00:00, 67.73it/s] 


In [241]:
i=0; j=1
_,ax = plt.subplots()
dat = amps_data_200V
ax.scatter(dat[::4,i], dat[::4,j])
ax.scatter(dat[2::4,i], dat[2::4,j])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [253]:
data0 = amps_data_20V_2[:,0]/scale*voltage/voltage0_20

dataPlus=data0[::4]
dataMinus=data0[2::4]

_, ax= plt.subplots(figsize=(9.5,4))
ax.scatter(np.arange(len(dataPlus)), dataPlus, label='X-axis, plus, 20V, YZ-spin-96kHz, with-height-fb')
ax.scatter(np.arange(len(dataMinus)), dataMinus, label='X-axis, minus, 20V, YZ-spin-96kHz, with-height-fb')
#ax.plot(np.arange(0,2500), np.arange(0,2500)*0, '--r')
ax.set(xlabel='Dataset', ylabel=r'$\epsilon$')
ax.legend()
# ax.set(xlim=(0,500))
print('mean: ', np.mean(dataPlus), 'std: ', np.std(dataPlus)/np.sqrt(len(dataPlus)))
print('mean: ', np.mean(dataMinus), 'std: ', np.std(dataMinus)/np.sqrt(len(dataMinus)))
print(np.mean(dataMinus)+np.mean(dataPlus), np.std(dataMinus+dataPlus)/np.sqrt(len(dataMinus)))
print(np.mean(dataMinus)*0.73+np.mean(dataPlus))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  0.001425129997187482 std:  0.0013165706121680274
mean:  0.0010682705408439604 std:  0.0012895637171104403
0.002493400538031442 0.001914201390862109
0.002204967492003573


In [195]:
z_plus = np.polyfit(np.arange(len(dataPlus)), dataPlus, 1)
z_minus = np.polyfit(np.arange(len(dataPlus)), dataMinus, 1)
ax.plot(np.arange(len(dataPlus)), np.arange(len(dataPlus))*z_plus[0]+z_plus[1], 'r--')
ax.plot(np.arange(len(dataPlus)), np.arange(len(dataPlus))*z_minus[0]+z_minus[1], 'r--')
print(z_plus, z_minus)

[-8.40716172e-07  3.33860906e-03] [-4.37670706e-07 -5.42992020e-03]


In [196]:
print(len(dataMinus), len(dataMinus))
dataPlus_noise=amps_data_200V[::4,1]
dataMinus_noise=amps_data_200V[2::4,1]
amps_data_200V_after_cuts = np.array([dataMinus[i]*0.73+dataPlus[i] for i in range(len(dataMinus)) if (dataPlus_noise[i]<1e-3)&(dataMinus_noise[i]<1e-3)])
len(amps_data_200V_after_cuts)

750 750


0

In [188]:
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

data_hist = amps_data_200V_after_cuts
range_pram = 0.02
_,ax = plt.subplots()
entries, bins, patches = ax.hist(data_hist, bins=29, range=(-range_pram, range_pram), label='20V-x, xz-spin-100V')
ax.set(xlabel=r'$\epsilon$')
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [189]:
## gaussian fit to the histogram

bins_fit =[(bins[i+1]-bins[i])/2+bins[i] for i in range(len(entries))]

for i,ee in enumerate([entries]):
    x = bins_fit
    y= ee

    n = len(x)                          #the number of data
    mean = sum(x*y)/n                   #note this correction
    sigma = sum(y*(x-mean)**2)/n        #note this correction

    def gaus(x,a,x0,sigma):
        return a*np.exp(-(x-x0)**2/(2*sigma**2))

    popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
    ax.plot(x,gaus(x,*popt),'ro:',label='fit')

    print("mean: ", popt[1], "std: ", np.sqrt(pcov)[1,1])
#     print("mean: ", popt[2], "std: ", np.sqrt(pcov)[2,2])

mean:  -0.003267608481785415 std:  0.0001257348612484117


In [191]:
_,ax = plt.subplots()
ax.errorbar(bins_fit, entries, yerr=np.sqrt(entries), fmt='o', label='data')
ax.plot(x,gaus(x,*popt),'r--',label='fit')
ax.legend()
print("mean: ", popt[1], "std: ", np.sqrt(pcov)[1,1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

mean:  -0.003267608481785415 std:  0.0001257348612484117


In [130]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 142
phase_calib = -0.0563
# phase_calib = -0.0442

def get_amp_single(i):
    
    fname = r'/data/new_trap/20210819/Bead1/Discharge/SingleResponse/Trek/plus_x/Discharge_'+str(i)+'.h5'
    fname = r'/data/new_trap/20210819/Bead1/Discharge/SingleResponse/Trek/plus_x_2f/Discharge_'+str(i)+'.h5'
#     fname = r'/data/new_trap/20210819/Bead1/Discharge/SingleResponse/Trek/minus_x/Discharge_'+str(i)+'.h5'
    bb = BeadDataFile.BeadDataFile(fname=fname)


    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}
    
    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, bb.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [131]:
amps_plus_x_2f = np.array(Parallel(n_jobs=20)(delayed(get_amp_single)(i) for i in tqdm(range(10))))

100%|██████████| 10/10 [00:00<00:00, 8160.12it/s]


In [39]:
factor = np.mean(amps_plus_x)/np.mean(amps_minus_x)
factor

0.7289756347706263

In [133]:
factor2 = np.mean(amps_plus_x_2f)/np.mean(amps_plus_x)
factor2

1.124106002733584

In [37]:
amps_plus_x, amps_minus_x

(array([-0.00299865, -0.00299193, -0.0030011 , -0.0030093 , -0.00300316,
        -0.00301009, -0.00300505, -0.0029996 , -0.00300229, -0.00300468]),
 array([-0.00408474, -0.00408243, -0.00409705, -0.00410182, -0.00412409,
        -0.00413091, -0.00412064, -0.00413625, -0.00414977, -0.00416141]))